In [8]:
#read json file
import json
with open('/Users/nelesuffo/Promotion/projects/TAAT/categories/human_actions/100898.json', 'r') as file:
    data = json.load(file)

In [21]:
data['annotations'][0]

{'categoryId': 'human_actions',
 'eventId': 'location',
 'fields': {'Type of Location': 'Livingroom'},
 'id': 'location_1745396774002',
 'time': 0,
 'type': 'start',
 'videoName': '100898.MP4'}

In [23]:
# Read the JSON file
import json

with open('/Users/nelesuffo/Promotion/projects/TAAT/categories/human_actions/100898.json', 'r') as file:
    data = json.load(file)

# Extract all 'id' and 'startAnnotationId' values from the annotations
ids = {annotation['id'] for annotation in data['annotations'] if 'id' in annotation}
start_annotation_ids = {annotation['startAnnotationId'] for annotation in data['annotations'] if 'startAnnotationId' in annotation}

# Find IDs that are only in 'id' or only in 'startAnnotationId'
only_in_ids = ids - start_annotation_ids
only_in_start_annotation_ids = start_annotation_ids - ids

# Print the results
print("IDs only in 'id':", only_in_ids)
print("IDs only in 'startAnnotationId':", only_in_start_annotation_ids)

IDs only in 'id': set()
IDs only in 'startAnnotationId': set()
